In [54]:
# uncomment these lines for one time install of required packages

# !conda install -c conda-forge geopy --yes 
# !pip install geocoder
# !pip install --upgrade pandas pgeocode
# !pip install pgeocode
# !conda install -c conda-forge folium=0.5.0 --yes
# !pip install selenium 

import pandas as pd                     # For Dataframes, plotting etc
import numpy as np                      # For Mathematical calculation 
import zipfile                          # For unzipping the web scraped files
import os                               # For correcting the file paths
import requests                         # For geting files and jsons
import json                             # For reading and wrangling json files
from bs4 import BeautifulSoup           # For scraping html data from websites
from pandas import json_normalize       # For formatting the jsons readable
import folium                           # For plotting maps
import matplotlib as plt                # For plotting charts
import matplotlib.cm as cm              # For functions to handle colormaps
import matplotlib.colors as colors      # For colour maps
import pgeocode                         # For getting lat and lon for postal codes
from geopy.geocoders import Nominatim   # For address into latitude and longitude values on maps
from sklearn.cluster import KMeans      # import k-means from clustering stage
import warnings                         # For ignoreing all warnings 
warnings.filterwarnings('ignore')

In [55]:
# Read the data from Wikipedia.
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
# Check how many tables do we find on the html
print(len(data))

3


In [56]:
# Find the correct table.
wiki = data[0]
wiki

,0,1,2,3,4,5,6,7,8
0,M1A Not assigned,M2A Not assigned,M3A North York (Parkwoods),M4A North York (Victoria Village),M5A Downtown Toronto (Regent Park / Harbourfront),M6A North York (Lawrence Manor / Lawrence Heig...,M7A Queen's Park (Ontario Provincial Government),M8A Not assigned,M9A Etobicoke (Islington Avenue)
1,M1B Scarborough (Malvern / Rouge),M2B Not assigned,M3B North York (Don Mills) North,M4B East York (Parkview Hill / Woodbine Gardens),"M5B Downtown Toronto (Garden District, Ryerson)",M6B North York (Glencairn),M7B Not assigned,M8B Not assigned,M9B Etobicoke (West Deane Park / Princess Gard...
2,M1C Scarborough (Rouge Hill / Port Union / Hig...,M2C Not assigned,M3C North York (Don Mills) South (Flemingdon P...,M4C East York (Woodbine Heights),M5C Downtown Toronto (St. James Town),M6C York (Humewood-Cedarvale),M7C Not assigned,M8C Not assigned,M9C Etobicoke (Eringate / Bloordale Gardens / ...
3,M1E Scarborough (Guildwood / Morningside / Wes...,M2E Not assigned,M3E Not assigned,M4E East Toronto (The Beaches),M5E Downtown Toronto (Berczy Park),M6E York (Caledonia-Fairbanks),M7E Not assigned,M8E Not assigned,M9E Not assigned
4,M1G Scarborough (Woburn),M2G Not assigned,M3G Not assigned,M4G East York (Leaside),M5G Downtown Toronto (Central Bay Street),M6G Downtown Toronto (Christie),M7G Not assigned,M8G Not assigned,M9G Not assigned
5,M1H Scarborough (Cedarbrae),M2H North York (Hillcrest Village),M3H North York (Bathurst Manor / Wilson Height...,M4H East York (Thorncliffe Park),M5H Downtown Toronto (Richmond / Adelaide / King),M6H West Toronto (Dufferin / Dovercourt Village),M7H Not assigned,M8H Not assigned,M9H Not assigned
6,M1J Scarborough (Scarborough Village),M2J North York (Fairview / Henry Farm / Oriole),M3J North York (Northwood Park / York University),M4J East York East Toronto (The Danforth East),M5J Downtown Toronto (Harbourfront East / Unio...,M6J West Toronto (Little Portugal / Trinity),M7J Not assigned,M8J Not assigned,M9J Not assigned
7,M1K Scarborough (Kennedy Park / Ionview / East...,M2K North York (Bayview Village),M3K North York (Downsview) East (CFB Toronto),M4K East Toronto (The Danforth West / Riverdale),M5K Downtown Toronto (Toronto Dominion Centre ...,M6K West Toronto (Brockton / Parkdale Village ...,M7K Not assigned,M8K Not assigned,M9K Not assigned
8,M1L Scarborough (Golden Mile / Clairlea / Oakr...,M2L North York (York Mills / Silver Hills),M3L North York (Downsview) West,M4L East Toronto (India Bazaar / The Beaches W...,M5L Downtown Toronto (Commerce Court / Victori...,M6L North York (North Park / Maple Leaf Park /...,M7L Not assigned,M8L Not assigned,M9L North York (Humber Summit)
9,M1M Scarborough (Cliffside / Cliffcrest / Scar...,M2M North York (Willowdale / Newtonbrook),M3M North York (Downsview) Central,M4M East Toronto (Studio District),M5M North York (Bedford Park / Lawrence Manor ...,M6M York (Del Ray / Mount Dennis / Keelsdale a...,M7M Not assigned,M8M Not assigned,M9M North York (Humberlea / Emery)


In [57]:
# Creating a function to loop through the html table and store the values in a seperate  lists
def get_first_three_chars_and_substring(wiki):
    # Create empty lists to store the results
    postalcode_list = []
    borough_list = []
    neighborhood_list = []

    # Iterate through each column in the DataFrame
    for col in wiki:
        # Retrieve the first 3 characters of each string in the column and store in a list
        col_values = wiki[col].astype(str).apply(lambda x: x[:3]).tolist()
        postalcode_list.append(col_values)

        # Retrieve the substring from the 4th place value till the bracket open symbol "("
        substr_values = wiki[col].astype(str).apply(lambda x: x[4:x.find(' (')] if ' (' in x else x[4:]).tolist()
        borough_list.append(substr_values)

        # Retrieve the values within the brackets
        bracket_values = wiki[col].astype(str).apply(lambda x: x[x.find('(')+1:x.find(')')] if '(' in x and ')' in x else '').tolist()
        neighborhood_list.append(bracket_values)

    return postalcode_list, borough_list, neighborhood_list

# Get the first 3 characters, substrings, and bracket values of each column and store in separate lists
postalcode_list, borough_list, neighborhood_list = get_first_three_chars_and_substring(wiki)

# Flatten the lists
postalcode_flat = [item for sublist in postalcode_list for item in sublist]
borough_flat = [item for sublist in borough_list for item in sublist]
neighborhood_flat = [item for sublist in neighborhood_list for item in sublist]

# Create a DataFrame
toronto = pd.DataFrame({
    'Postal Code': postalcode_flat,
    'Borough': borough_flat,
    'Neighborhood': neighborhood_flat
})

# Drop rows where the column "Borough" has the value "Not assigned"
toronto = toronto[toronto['Borough'] != 'Not assigned']

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Display the resulting DataFrame
toronto.sort_values(['Postal Code']).sample(10)


,Postal Code,Borough,Neighborhood
94,M5T,Downtown Toronto,Kensington Market / Chinatown / Grange Park
107,M6K,West Toronto,Brockton / Parkdale Village / Exhibition Place
175,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
80,M5A,Downtown Toronto,Regent Park / Harbourfront
86,M5J,Downtown Toronto,Harbourfront East / Union Station / Toronto Is...
30,M2N,North York,Willowdale
104,M6G,Downtown Toronto,Christie
61,M4B,East York,Parkview Hill / Woodbine Gardens
102,M6C,York,Humewood-Cedarvale
87,M5K,Downtown Toronto,Toronto Dominion Centre / Design Exchange


In [58]:
# Create a Nominatim instance for Canada
nomi = pgeocode.Nominatim('CA')

# Create new columns for latitude and longitude
toronto['Latitude'] = None
toronto['Longitude'] = None

# Iterate over each row to get the latitude and longitude
for index, row in toronto.iterrows():
    location = nomi.query_postal_code(row['Postal Code'])
    toronto.at[index, 'Latitude'] = location.latitude
    toronto.at[index, 'Longitude'] = location.longitude

#Display setting
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None) 
pd.set_option('display.width',500)


# Display the updated DataFrame
display(toronto.describe())
display(toronto.info())
display(toronto)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
count,103,103,103,102.0000,102.0000
unique,103,15,98,98.0000,97.0000
top,M1B,North York,Downsview,43.7334,-79.3823
freq,1,24,4,2.0000,3.0000


<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, 1 to 176
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Postal Code   103 non-null    object
 1   Borough       103 non-null    object
 2   Neighborhood  103 non-null    object
 3   Latitude      102 non-null    object
 4   Longitude     102 non-null    object
dtypes: object(5)
memory usage: 8.9+ KB


None

,Postal Code,Borough,Neighborhood,Latitude,Longitude
1,M1B,Scarborough,Malvern / Rouge,43.8113,-79.193
2,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.7878,-79.1564
3,M1E,Scarborough,Guildwood / Morningside / West Hill,43.7678,-79.1866
4,M1G,Scarborough,Woburn,43.7712,-79.2144
5,M1H,Scarborough,Cedarbrae,43.7686,-79.2389
6,M1J,Scarborough,Scarborough Village,43.7464,-79.2323
7,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.7298,-79.2639
8,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.7122,-79.2843
9,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.7247,-79.2312
10,M1N,Scarborough,Birch Cliff / Cliffside West,43.6952,-79.2646


In [59]:
# Does Toronto have 103 postal codes? lets check another website for counts of postal codes in Toronto city
# # URL of the file to be downloaded
url = 'https://download.geonames.org/export/zip/CA_full.csv.zip'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save the content to a file
    zip_file_path = 'CA_full.csv.zip'
    with open(zip_file_path, 'wb') as file:
        file.write(response.content)
    print('File downloaded successfully!')

    # Unzip the file
    extract_path = 'extracted_files'
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print('File unzipped successfully!')

    # Optionally, remove the zip file after extraction
    os.remove(zip_file_path)
else:
    print('Failed to download the file.')

File downloaded successfully!
File unzipped successfully!


In [60]:
# Read the file
file_path = 'extracted_files/CA_full.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

# Filter lines where the text after "CA" and the tab starts with "M"
filtered_lines = [line for line in lines if line.split('\t')[1].startswith('M')]

# Create a DataFrame from the filtered lines
data = [line.strip().split('\t') for line in filtered_lines]
df = pd.DataFrame(data, columns=['Country', 'Postal Code', 'Place Name', 'Province', 'Province Abbreviation', 'Column6', 'Column7', 'Column8', 'Latitude', 'Longitude', 'Column11', 'Column12'])

# Split the postal code into two parts
df['Postal Code Part 1'] = df['Postal Code'].str[:3]
df['Postal Code Part 2'] = df['Postal Code'].str[3:]

# Count distinct values in "Postal Code Part 1"
distinct_count = df['Postal Code Part 1'].nunique()

# Display the count
print(f"Count of distinct values of Toronto Postal codes is: {distinct_count}")


Count of distinct values of Toronto Postal codes is: 103


In [61]:
rows_with_null = toronto[toronto.isnull().any(axis=1)]
rows_with_null

,Postal Code,Borough,Neighborhood,Latitude,Longitude
132,M7R,Mississauga Canada Post Gateway Processing Centre,Enclave of L4W,NaN,NaN


In [62]:
toronto_df = toronto.dropna(subset=['Latitude'])
toronto_df = toronto_df[~toronto_df['Neighborhood'].str.startswith('Enclave')] 
toronto_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 1 to 176
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Postal Code   100 non-null    object
 1   Borough       100 non-null    object
 2   Neighborhood  100 non-null    object
 3   Latitude      100 non-null    object
 4   Longitude     100 non-null    object
dtypes: object(5)
memory usage: 4.7+ KB


In [63]:
# Creating a Neighbourhood specific dataframe
toronto_df_n = pd.DataFrame(toronto_df)

# Split the strings in the third column and expand into separate rows
toronto_df_n['Neighborhood'] = toronto_df_n['Neighborhood'].str.split('/')
toronto_df_n = toronto_df_n.explode('Neighborhood').reset_index(drop=True)

toronto_df_n.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Postal Code   212 non-null    object
 1   Borough       212 non-null    object
 2   Neighborhood  212 non-null    object
 3   Latitude      212 non-null    object
 4   Longitude     212 non-null    object
dtypes: object(5)
memory usage: 8.4+ KB


In [64]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [65]:
# Path to the downloaded population file
path_population = 'extracted_files/population_data_2021.csv'

# Read the CSV file into a DataFrame
population = pd.read_csv(path_population)

# Filter rows where the values in the "Geographic name" column start with "M"
population_df = population[population['Geographic name'].str.startswith('M')]

# Display the filtered DataFrame
population_df = population_df.rename(columns={'Geographic name':'Postal Code'}).astype(str)
population_df['Population, 2021'] = population_df['Population, 2021'].str.replace(',', '').astype(int)
population_df['Total private dwellings, 2021'] = population_df['Total private dwellings, 2021'].str.replace(',', '').astype(int)
population_df['Private dwellings occupied by usual residents, 2021'] = population_df['Private dwellings occupied by usual residents, 2021'].str.replace(',', '').astype(int)
population_df = population_df.reset_index(drop=True)
print(population_df.dtypes)
population_df.head()

Postal Code                                            object
Population, 2021                                        int64
Total private dwellings, 2021                           int64
Private dwellings occupied by usual residents, 2021     int64
dtype: object


,Postal Code,"Population, 2021","Total private dwellings, 2021","Private dwellings occupied by usual residents, 2021"
0,M1B,65555,21561,20563
1,M1C,35642,11724,11396
2,M1E,48033,18085,17320
3,M1G,30894,10791,10313
4,M1H,23964,9021,8632


In [66]:
# As we have 97 postal codes in totonto_df and 97 in population_df, let us find out the difference
missing_postalcodes = toronto_df[~toronto_df['Postal Code'].isin(population_df['Postal Code'])]
missing_postalcodes

,Postal Code,Borough,Neighborhood,Latitude,Longitude
87,M5K,Downtown Toronto,Toronto Dominion Centre / Design Exchange,43.6469,-79.3823
88,M5L,Downtown Toronto,Commerce Court / Victoria Hotel,43.6492,-79.3823
97,M5X,Downtown Toronto,First Canadian Place / Underground city,43.6492,-79.3823


In [67]:
# Looking for more duplicates of latitude and longitude 
Lat_lon_duplicates = toronto_df[toronto_df.duplicated(subset=['Latitude', 'Longitude'], keep=False)]
Lat_lon_duplicates

,Postal Code,Borough,Neighborhood,Latitude,Longitude
88,M5L,Downtown Toronto,Commerce Court / Victoria Hotel,43.6492,-79.3823
97,M5X,Downtown Toronto,First Canadian Place / Underground city,43.6492,-79.3823
